In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random

from scipy import sparse
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import plotnine
from plotnine import *

In [ ]:
# 데이터 불러오기 
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')
meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')

### 데이터 전처리

In [ ]:
# log_time 컬럼 중, 이상치 변환
def time_preprocessing(t):
    s = int(str(t)[-2:])
    m = int(str(t)[-4:-2])
    h = int(str(t)[-6:-4])
    if s >= 60:
        m += s//60
        s %= 60
    if m >= 60:
        h += m//60
        m %= 60
    if h >= 24:
        h = 23
        m = 59
        s = 59
    return str(t)[:-6] + str(h).zfill(2) + str(m).zfill(2) + str(s).zfill(2)

for df in [history_df, watch_df, search_df]:
    for col in ["ss_id", "log_time"]:
        df[col] = df[col].apply(lambda x: time_preprocessing(x))
        df[col] = df[col].apply(lambda x: datetime.datetime.strptime(str(x),"%Y%m%d%H%M%S"))

In [ ]:
# history_df와 search_df를 merge... 단, log_time이 다름에 주의해야함.
temp_h = history_df.copy()
temp_s = search_df.copy()

# join을 위해 년월일 시까지만 고려.
temp_h["log_for_join"] = temp_h["log_time"].apply(lambda x: datetime.datetime.strftime(x, "%Y%m%d%H"))
temp_s["log_for_join"] = temp_s["log_time"].apply(lambda x: datetime.datetime.strftime(x, "%Y%m%d%H"))
temp_s.drop(columns=["log_time"], inplace=True)

merged_df = pd.merge(temp_h, temp_s, how='left', 
                     on=["profile_id","ss_id", "log_for_join", "album_id"])
merged_df

In [ ]:
# merged_df와 watch_df를 merge ... 단, log_time이 다름.
temp_m = merged_df.copy()
temp_w = watch_df.copy()

temp_m = pd.merge(temp_m, temp_w, how='inner',
                  on=["profile_id","ss_id", "album_id"])
temp_m

In [ ]:
# log_diff 와 watch_time 간, 차이가 큰 경우를 중복데이터 및 이상치로 판단하여 해당 rows 제거.
# 일단 log_diff가 양수인 경우만 고려.
temp_m = temp_m[temp_m["log_diff"] >= 0]
temp_m

In [ ]:
# 콘텐츠 전체 길이에 대한 시청 비율 컬럼을 추가
temp_m["time_ratio"] = temp_m["watch_time"] / temp_m["total_time"]

In [ ]:
# 데이터 전처리 (중복제거) 
# 참고 : drop_duplicates의 subset을 무엇으로 구성하냐에 따라서 제거되는 항목들이 다름 
# ex) 'profile_id', 'album_id' : 중복된 시청이력 모두 제거 / 'profile_id', 'album_id', 'log_time' : 같은 시간에 시청한 이력만 제거 
data = history_df[['profile_id', 'log_time', 'album_id']].drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True)
data['rating'] = 1

# Matrix 형태로 변환 
train = train.to_numpy()
matrix = sparse.lil_matrix((cfg.n_users, cfg.n_items))
for (p, _, i, r) in tqdm(train):
    matrix[p, i] = r

In [ ]:
#기존과 다르게 cast에 대한 정보 추가

le = LabelEncoder()
meta_df['cast_1'] = le.fit_transform(meta_df['cast_1'])
item_features['cast_1'] = meta_df[['cast_1']].to_dict()['cast_1']

cfg.n_cast = meta_df['cast_1'].nunique()
cfg.n_continuous_feats = 1

In [ ]:
# 새로 작성한 코드만 

cast_embedding_mlp = self.cast_embeddig(feats[2])
        ###########################################################################
input_feature = torch.cat((user_embedding_mlp, item_embedding_mlp, genre_embedding_mlp, cast_embedding_mlp, feats[0].unsqueeze(1)), -1)
mlp_output = self.mlp_layers(input_feature)
        
output = torch.cat([mlp_output, mf_output], dim=-1)
output = self.affine_output(output).squeeze(-1)

features = []
for item_id in np.concatenate([pos_item_ids, neg_item_ids]): 
        features.append(item_features['cast_1'][item_id])
        
UIdataset[user_id].append(np.array(features))